In [ ]:
# Colab: Phi-3 + French NER PII detection & anonymization pipeline
# Copy entire cell blocks into a new Google Colab notebook and run sequentially.


In [ ]:
# 0) Environment: install dependencies
# NOTE: model card đề xuất dùng transformers dev & specific versions.
!pip install -q git+https://github.com/huggingface/transformers@main  # dev transformers (per model card)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # GPU build; may change per Colab runtime
!pip install -q accelerate
!pip install -q datasets
!pip install -q sentencepiece
!pip install -q faker
!pip install -q french-lefff  # optional for French morphology (if needed)
!pip install -q transformers[torch]


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement french-lefff (from versions: none)
ERROR: No matching distribution found for french-lefff


In [ ]:
# 1) Imports
import os, re, json, uuid
from pprint import pprint

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# NER
from transformers import AutoModelForTokenClassification, AutoTokenizer as HFTokenizer
from transformers import pipeline as hf_pipeline

# Synthetic data
from faker import Faker
fake = Faker('fr_FR')


In [ ]:
# 2) Configure Hugging Face token (if needed for private models or higher rate)
# (set as environment variable in Colab: replace YOUR_HF_TOKEN with your token)
# os.environ['HUGGINGFACE_HUB_TOKEN'] = "YOUR_HF_TOKEN"

# 3) Load French NER model (CamemBERT NER)
ner_model_name = "Jean-Baptiste/camembert-ner-with-dates"  # good French NER variant
ner_tokenizer = HFTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner = hf_pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")
print("NER pipeline ready.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

CamembertForTokenClassification LOAD REPORT from: Jean-Baptiste/camembert-ner-with-dates
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


NER pipeline ready.


In [ ]:
# 4) PII detection helper (NER + regex rules)
PII_REGEX = {
    "EMAIL": r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",
    "IBAN": r"[A-Z]{2}[0-9]{2}[A-Z0-9]{1,30}",  # simplistic IBAN-ish pattern - refine in prod
    "MONEY": r"\b[0-9]{1,3}(?:[ .,][0-9]{3})*(?:€|\s?EUR)\b|\b[0-9]+(?:\.[0-9]{2})?\s?€\b",
    "DATE": r"\b(?:\d{1,2}\s(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)|\d{1,2}/\d{1,2}/\d{2,4}|\d{4})\b",
    # more rules can be added: phone, SIREN, contract numbers...
}

def regex_pii_find(text):
    hits = []
    for label, pattern in PII_REGEX.items():
        for m in re.finditer(pattern, text, flags=re.IGNORECASE):
            hits.append({"entity": label, "start": m.start(), "end": m.end(), "text": m.group(0)})
    return hits

def ner_pii_find(text):
    entities = ner(text)
    # output: list of dicts with 'entity_group', 'word', 'start', 'end'
    return [{"entity": e['entity_group'], "start": e['start'], "end": e['end'], "text": e['word']} for e in entities]


In [ ]:
# 5) Anonymization / pseudonymization utilities
# We will replace detected spans with tokens like <PER_1>, <ORG_1> and optionally keep encrypted mapping.

from cryptography.fernet import Fernet
# create key for mapping encryption (in prod store in KMS)
# key = Fernet.generate_key()
# f = Fernet(key)
# For demo we skip encryption but include placeholder
mapping_store = {}  # map token -> original (in prod store encrypted)
token_counters = {}

def get_token(entity_label):
    token_counters.setdefault(entity_label, 0)
    token_counters[entity_label] += 1
    return f"<{entity_label}_{token_counters[entity_label]}>"

def anonymize_text(text):
    # 1) collect spans from NER and regex
    ner_spans = ner_pii_find(text)
    regex_spans = regex_pii_find(text)
    spans = ner_spans + regex_spans
    # merge and sort by start
    spans_sorted = sorted(spans, key=lambda x: x['start'])
    # build anonymized text
    out = []
    last = 0
    for s in spans_sorted:
        if s['start'] < last:
            continue  # overlapping/covered
        out.append(text[last:s['start']])
        token = get_token(s['entity'])
        mapping_store[token] = s['text']
        out.append(token)
        last = s['end']
    out.append(text[last:])
    return "".join(out), mapping_store

# quick test on your example text (assign to variable `legal_text` below)


In [ ]:
# 6) Load Phi-3 (option A: local on-GPU - may not fit on Colab GPU)
model_name = "microsoft/Phi-3-mini-128k-instruct"

use_local_phi3 = False  # set True if your GPU/runtime can handle ~3.8B model

if use_local_phi3:
    print("Loading Phi-3 local (may need large GPU RAM).")
    tokenizer_phi = AutoTokenizer.from_pretrained(model_name)
    model_phi = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )
    gen_pipe = pipeline("text-generation", model=model_phi, tokenizer=tokenizer_phi, device_map="auto")
else:
    # Option B: use Hugging Face Inference API or Azure inference
    # Example: use HF Inference via small prompt: requires HF token and inference API call
    from transformers import TextGenerationPipeline
    print("Using lightweight local model for testing LLM prompts (fallback).")
    # We'll use a small builtin model for demo prompt testing, and show how to call Phi-3 via remote API.
    demo_model = "gpt2"  # demo only
    tokenizer_phi = AutoTokenizer.from_pretrained(demo_model)
    model_phi = AutoModelForCausalLM.from_pretrained(demo_model)
    gen_pipe = pipeline("text-generation", model=model_phi, tokenizer=tokenizer_phi)


Using lightweight local model for testing LLM prompts (fallback).


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from cryptography.fernet import Fernet
# For demo we create a key; in prod store key in Vault
FERNET_KEY = Fernet.generate_key()
fernet = Fernet(FERNET_KEY)

mapping_store = {}  # mapping token -> original (demo). In prod store encrypted and access-controlled
token_counters = {}

def get_token(entity_label):
    token_counters.setdefault(entity_label, 0)
    token_counters[entity_label] += 1
    return f"<{entity_label}_{token_counters[entity_label]}>"

def anonymize_text(text):
    ner_spans = ner_pii_find(text)
    regex_spans = regex_pii_find(text)
    spans = ner_spans + regex_spans
    spans_sorted = sorted(spans, key=lambda x: x['start'])
    out = []
    last = 0
    for s in spans_sorted:
        if s['start'] < last:
            continue  # skip overlaps already covered
        out.append(text[last:s['start']])
        token = get_token(s['entity'])
        # store encrypted original
        cipher = fernet.encrypt(s['text'].encode()).decode()
        mapping_store[token] = {"cipher": cipher, "entity": s['entity'], "original_preview": s['text'][:50]}
        out.append(token)
        last = s['end']
    out.append(text[last:])
    return "".join(out), mapping_store


In [ ]:
# 7) Demo: anonymize and then ask model to summarize (workflow)
legal_text = """
Cabinet AFG Conseil – Note fiscale n° 2025/034
Client : SCI Horizon Invest
Date : 15 mai 2025
Objet : Analyse des conséquences fiscales de la cession d’un immeuble détenu par une SCI soumise à l’Impôt sur les Sociétés (IS)
...
Conseiller en charge du dossier :
Me Sophie LAMGFT, Expert fiscal agréé
"""

# The anonymize_text function is defined in a previous cell (cjHN_D3aBXGY).
# The error occurs because the sorted() function attempts to compare an integer with a NoneType
# when one of the detected PII spans has a 'start' key with a None value.
# To fix this, the anonymize_text function needs to be modified to filter out invalid spans
# where 'start' or 'end' are not integers.
#
# Due to the constraint to only modify the *selected cell*,
# I am re-defining the corrected anonymize_text function here.
# In a real scenario, you would modify the definition in cell 'cjHN_D3aBXGY'.

def anonymize_text(text):
    # 1) collect spans from NER and regex
    ner_spans = ner_pii_find(text)
    regex_spans = regex_pii_find(text)
    spans = ner_spans + regex_spans

    # Filter out any spans where 'start' or 'end' might be None or not an integer.
    # This prevents TypeError during sorting if an entity's start/end is invalid.
    filtered_spans = []
    for s in spans:
        if isinstance(s.get('start'), int) and isinstance(s.get('end'), int):
            filtered_spans.append(s)
        # Optional: Print a warning for skipped invalid spans
        # else:
        #     print(f"Warning: Skipping invalid span due to non-integer start/end: {s}")

    # merge and sort by start
    spans_sorted = sorted(filtered_spans, key=lambda x: x['start'])

    # build anonymized text
    out = []
    last = 0
    # Reset token_counters and mapping_store for each call if new numbering/mapping is desired
    # They are defined as global in the original notebook setup.
    global token_counters
    global mapping_store
    token_counters = {}
    mapping_store = {}

    for s in spans_sorted:
        if s['start'] < last:
            continue  # overlapping/covered
        out.append(text[last:s['start']])
        token = get_token(s['entity'])
        mapping_store[token] = s['text']
        out.append(token)
        last = s['end']
    out.append(text[last:])
    return "".join(out), mapping_store

anonymized_text, mapping = anonymize_text(legal_text)
print("=== Anonymized ===")
print(anonymized_text)
print("\n=== Mapping (sample) ===")
pprint(list(mapping.items())[:10])

=== Anonymized ===

Cabinet AFG Conseil – Note fiscale n° <DATE_1>/034
Client : SCI Horizon Invest
Date : <DATE_2> <DATE_3>
Objet : Analyse des conséquences fiscales de la cession d’un immeuble détenu par une SCI soumise à l’Impôt sur les Sociétés (IS)
...
Conseiller en charge du dossier :
Me Sophie LAMGFT, Expert fiscal agréé


=== Mapping (sample) ===
[('<DATE_1>', '2025'), ('<DATE_2>', '15 mai'), ('<DATE_3>', '2025')]


In [ ]:
# 8) Prompt the (demo) LLM to summarize the anonymized content (replace with Phi-3 generation when possible)
prompt = f"""<|system|>
You are a helpful assistant. Summarize the fiscal consequences in 3 bullet points from the text below. Do NOT attempt to de-anonymize tokens.
<|end|>
<|user|>
{textwrap.dedent(anonymized_text)}
<|end|>"""

out = gen_pipe(prompt, max_new_tokens=200, do_sample=False, temperature=0.0)
print(out[0]['generated_text'])


In [ ]:
# 9) Synthetic data generation (generator for finance notes)
def gen_synthetic_note():
    company = fake.company()
    seller = fake.company()
    price_purchase = f"{fake.random_int(500000, 5000000):,} €".replace(",", " ")
    date_acq = fake.date_between(start_date='-10y', end_date='-1y').strftime("%d %B %Y")
    price_sell = f"{fake.random_int(1000000, 8000000):,} €".replace(",", " ")
    note = f"Cabinet {fake.last_name()} Conseil – Note fiscale n° {fake.random_int(100,999)}/{fake.random_int(2020,2025)}\nClient : {company}\nDate : {fake.date_between(start_date='-365d', end_date='today').strftime('%d %B %Y')}\nObjet : Analyse des conséquences fiscales de la cession d’un immeuble détenu par {seller}\n\n1. Contexte\nLe {seller} envisage de céder un immeuble acquis en {date_acq} pour un prix d’achat de {price_purchase}.\nLe prix de cession envisagé est de {price_sell}.\n\n..."
    return note

# generate 5 synthetic notes
synth_data = [gen_synthetic_note() for _ in range(5)]
print(synth_data[0])



---

# 2) Giải thích chi tiết từng bước / Tại sao như vậy

1. **Detecter (PII detection)** — bước đầu bắt buộc:
   - Kết hợp **NER model (CamemBERT-NER)** cho entities tiếng Pháp (PER, ORG, LOC, DATE) và **regex** cho các pattern domain-specific (IBAN, số tài khoản, số hợp đồng, money amounts).  
   - Lý do: LLM có thể giúp nhận diện ngữ cảnh nhưng thường **hay hallucinate**; encoder models (BERT/CamemBERT) fine-tuned cho token classification thường ổn định hơn cho NER. :contentReference[oaicite:2]{index=2}

2. **Remplacer / Anonymiser / Pseudonymize**:
   - Thay thế bằng token như `<PER_1>` hoặc bằng pseudonym (ví dụ: map thành `Client_042`) và lưu mapping an toàn (mã hóa và lưu trong KMS) nếu cần khôi phục.  
   - Giữ log metadata (request_id, policy_id, model route) để audit mà không lưu dữ liệu gốc.

3. **LLM (Phi-3) cho task nghiệp vụ**:
   - Sau khi dữ liệu đã được anonymized, gửi cho LLM để thực hiện summarization, classification, Q&A, extraction. LLM có ưu thế về reasoning/long-context (Phi-3 128k rất mạnh cho docs dài). :contentReference[oaicite:3]{index=3}
   - Nhắc lại: không gửi raw PII ra provider cloud nếu policy forbids → route đến on-prem Phi-3 hoặc sử dụng RAG/local retrieval.

4. **Synthetic data**:
   - Vì dữ liệu thật nhạy cảm, tạo **synthetic dataset** (Faker + templates + perturbations) để huấn luyện/finetune detector hoặc đánh giá.  
   - Bạn có thể thêm perturbations: misspellings, varying formats (15 mai 2025 / 15/05/2025), locale differences, abbreviations.

5. **Evaluation**:
   - Đo precision/recall/F1 của PII detection trên dataset synthetic + small labeled holdout.  
   - Đo *utility loss* (tức là performance của downstream task như summarization) khi áp anonymization vs raw (but raw maybe forbidden).

---

# 3) Lựa chọn mô hình: BERT vs LLM (kết luận ngắn)

- **Detection (NER/PII):** ưu tiên **token-classification models (CamemBERT/BERT-based)** fine-tuned cho NER trong tiếng Pháp. Tại sao: độ ổn định, predictability, nhẹ hơn, ít hallucinate. :contentReference[oaicite:4]{index=4}  
- **Anonymization logic:** rule-based + deterministic (regex + mapping) tốt cho audit và GDPR.  
- **Downstream tasks (summaries / insights):** dùng **Phi-3** (hoặc các LLM) trên dữ liệu đã anonymized. Phi-3 có lợi thế với context dài. :contentReference[oaicite:5]{index=5}

---

# 4) Lộ trình nghiên cứu / experimental plan (gợi ý cho thèse CIFRE)

1. **P0 — State of the art**: survey NER French, privacy-preserving NLP, synthetic data methods. (1–2 tháng)  
2. **P1 — Prototype detection & anonymization**: NER + regex + mapping + small UI to inspect mappings. (2 tháng)  
3. **P2 — Synthetic dataset & benchmarks**: build synthetic corpus (10k docs), create labeled PII for evaluation. (2 tháng)  
4. **P3 — Integration with Phi-3 & evaluation**: measure downstream utility (summarization quality, hallucinations, leakage risk). (3 tháng)  
5. **P4 — Advanced methods**: context-aware anonymization (learned policy), differential privacy experiments, confidential computing pilot. (rest of thesis)  

Metrics to track: NER F1, downstream BLEU/ROUGE or human eval, number of PII leaks (manual + automated), latency, throughput.

---

# 5) Vấn đề practical & cảnh báo

- **Phi-3 model card yêu cầu:** sử dụng `trust_remote_code=True` và/hoặc transformers dev build để load. Nếu bạn đưa model ra production/on-prem, kiểm tra licensing and safety notes. :contentReference[oaicite:6]{index=6}  
- **GDPR:** giữ mapping encrypted, support DSAR (right to erasure) — có thể implement "delete mapping" để xóa mọi khả năng khôi phục.  
- **Evaluation of leakage:** test prompts adversarial để cố gắng ép model tái tạo PII — đánh giá rủi ro.

---

# 6) Thực thi nhanh: test với đoạn văn bạn cung cấp

Bạn có thể copy đoạn văn (note fiscale) vào biến `legal_text` ở cell trên và chạy. Pipeline sẽ:

- phát hiện entities (PER, ORG, DATE) via camembert-ner  
- phát hiện amounts via regex (e.g. `2 500 000 €`)  
- trả về text đã anonymize và mapping (lưu mapping local cho dev; production: store encrypted in Vault/KMS)

---

# 7) Tài liệu & tham khảo (đã dùng)
- Phi-3 Mini-128K Instruct model card — Hugging Face (thông tin model, requirement sử dụng transformers dev). :contentReference[oaicite:7]{index=7}  
- Jean-Baptiste camembert-ner (French NER model). :contentReference[oaicite:8]{index=8}




In [ ]:
# app.py (simplified)
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
import uuid, json, re
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from cryptography.fernet import Fernet
import os

app = FastAPI()

# Demo: generate key (in prod store in Vault/KMS)
FERNET_KEY = os.environ.get("FERNET_KEY") or Fernet.generate_key()
fernet = Fernet(FERNET_KEY)

# load NER model
ner_name = "Jean-Baptiste/camembert-ner-with-dates"
ner = pipeline("ner", model=ner_name, tokenizer=ner_name, aggregation_strategy="simple")

PII_REGEX = {
    "EMAIL": r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",
    "MONEY": r"\b[0-9]{1,3}(?:[ .,][0-9]{3})*(?:€|\s?EUR)\b",
    # Add IBAN, SIREN, etc...
}

# in-memory mapping (demo). In prod: encrypted DB
mapping_db = {}

class InferRequest(BaseModel):
    client_id: str
    purpose: str
    retention_policy_id: str = "rp-30d"
    payload: dict

@app.post("/v1/infer")
async def infer(req: InferRequest):
    text = req.payload.get("text", "")
    if not text:
        raise HTTPException(status_code=400, detail="Missing payload.text")

    request_id = str(uuid.uuid4())

    # 1. Detect PII via NER
    ner_entities = ner(text)
    spans = []
    for e in ner_entities:
        spans.append({"start": e["start"], "end": e["end"], "entity": e["entity_group"], "text": e["word"]})

    # 2. Regex detection
    for label, pat in PII_REGEX.items():
        for m in re.finditer(pat, text, flags=re.IGNORECASE):
            spans.append({"start": m.start(), "end": m.end(), "entity": label, "text": m.group(0)})

    # 3. Merge spans, create tokens and store mapping
    spans_sorted = sorted(spans, key=lambda x: x["start"])
    anonymized = []
    last = 0
    counter = {}
    for s in spans_sorted:
        if s["start"] < last:
            continue
        anonymized.append(text[last:s["start"]])
        counter.setdefault(s["entity"], 0)
        counter[s["entity"]] += 1
        token = f"<{s['entity']}_{counter[s['entity']]}>"
        # encrypt original
        cipher = fernet.encrypt(s["text"].encode()).decode()
        mapping_db[token] = {"cipher": cipher, "entity": s["entity"], "request_id": request_id}
        anonymized.append(token)
        last = s["end"]
    anonymized.append(text[last:])

    sanitized_text = "".join(anonymized)

    # 4. policy decision (demo: always route to onprem)
    route = "onprem"

    # 5. call model (placeholder)
    model_response = {"summary": f"SUMMARY of sanitized text (len {len(sanitized_text)})"}

    # 6. audit log (don't store raw text)
    audit_record = {
        "request_id": request_id,
        "client_id": req.client_id,
        "route": route,
        "pii_spans": len(spans_sorted),
    }
    print("AUDIT:", json.dumps(audit_record))

    return {"request_id": request_id, "sanitized_text": sanitized_text, "model_response": model_response}
